In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString
from shapely.ops import nearest_points
from tqdm import tqdm

In [3]:
# Carregar os dados
tracking_GPS = gpd.read_file("C:/Users/gus_h/OneDrive/Área de Trabalho/Dissertação/1 - CAMINHOS RODOFLUVIAIS/VERTICES TRAJETOS CENSO - 102033/VERTICES TRAJETOS CENSO - 102033.shp")
poligonos = gpd.read_file("C:/Users/gus_h/OneDrive/Área de Trabalho/Dissertação/1 - CAMINHOS RODOFLUVIAIS/TERRA FIRME BAIXO TOCANTINS - PARTES SIMPLES - 102033/TERRA FIRME BAIXO TOCANTINS - PARTES SIMPLES - 102033.shp")

In [5]:
# Definir um CRS projetado, por exemplo, UTM
crs_projetado = "EPSG:31982"  # 
pontos = tracking_GPS.to_crs(crs_projetado)
poligonos = poligonos.to_crs(crs_projetado)
poligonos = poligonos.drop(columns=['AREA_KM2', 'osm_id', 'code', 'fclass', 'name']) #retirando colunas indesejadas


print(poligonos)

      CD_MUN NM_MUN SIGLA_UF   ID  \
0    1501204  Baião       PA    0   
1    1501204  Baião       PA    1   
2    1501204  Baião       PA    2   
3    1501204  Baião       PA    3   
4    1501204  Baião       PA    4   
..       ...    ...      ...  ...   
615  1501204  Baião       PA  615   
616  1501204  Baião       PA  616   
617  1501204  Baião       PA  617   
618  1501204  Baião       PA  618   
619  1501204  Baião       PA  619   

                                              geometry  
0    POLYGON ((654069.692 9611078.066, 654068.268 9...  
1    POLYGON ((640683.564 9664624.821, 640728.876 9...  
2    POLYGON ((640767.034 9664727.257, 640762.498 9...  
3    POLYGON ((640411.055 9665112.566, 640413.404 9...  
4    POLYGON ((640152.799 9665481.197, 640113.646 9...  
..                                                 ...  
615  POLYGON ((713623.633 9799976.032, 713587.844 9...  
616  POLYGON ((713176.73 9800105.451, 713118.098 98...  
617  POLYGON ((710252.409 9801972.22, 7102

In [7]:
def encontrar_poligono_mais_proximo(ponto, poligonos):
    if ponto.is_empty:  # Verifica se o ponto não está vazio
        return None, None, None
    
    distancias = poligonos.geometry.distance(ponto) # Calcula as distâncias entre o ponto e todos os polígonos
    
    indice_mais_proximo = distancias.idxmin()  # Encontra o índice do polígono mais próximo
    poligono_mais_proximo = poligonos.geometry[indice_mais_proximo] # Obtém o polígono mais próximo
    
    id_poligono_mais_proximo = poligonos['ID'].iloc[indice_mais_proximo]  # Obtém o ID do polígono mais próximo
    
    ponto_mais_proximo = nearest_points(ponto, poligono_mais_proximo)[1] # Calcula o ponto no polígono que esteja mais próximo da coordenada GPS
    
    linha = LineString([ponto, ponto_mais_proximo]) # Cria uma linha entre o ponto e o ponto mais próximo do polígono
    
    # Cria um GeoDataFrame para armazenar a linha e o ID do polígono
    linha_com_id = gpd.GeoDataFrame(geometry=[linha], data={'id_poligono': [id_poligono_mais_proximo]})
     
    return linha_com_id

In [9]:
# Inicializa listas para armazenar os resultados
linhas_resultantes = []
ids_proximos = []

In [11]:
# Itera sobre cada ponto e armazena os resultados
for index, row in tqdm(pontos.iterrows(), total=pontos.shape[0]):
    ponto_geom = row.geometry  # Acessa a geometria do ponto
    
    if ponto_geom is not None:  # Verifica se a geometria do ponto é válida
        resultado = encontrar_poligono_mais_proximo(ponto_geom, poligonos)  # Chama a função
        
        if resultado is not None:  # Verifica se o resultado não é None
            linha = resultado.geometry.values[0]  # Acessa a geometria da linha
            linhas_resultantes.append(linha)  # Armazena a geometria da linha
            
            id_poligono = resultado['id_poligono'].values[0]  # Acessa o ID do polígono
            ids_proximos.append(id_poligono)  # Armazena o ID do polígono

100%|████████████████████████████████████████████████████████████████████████████| 41030/41030 [09:08<00:00, 74.82it/s]


In [13]:
# Cria um DataFrame com as linhas e os IDs dos polígonos
resultado_df = gpd.GeoDataFrame({
    'linha': linhas_resultantes,
    'id_poligono': ids_proximos
})

In [15]:
# Define a coluna de geometria
resultado_df.set_geometry('linha', inplace=True)

# Define o CRS
resultado_df.set_crs(epsg=31982, inplace=True)

,linha,id_poligono
0,"LINESTRING (652854.686 9656206.853, 652854.686...",142
1,"LINESTRING (652852.045 9656259.164, 652830.221...",142
2,"LINESTRING (652843.613 9656271.315, 652820.691...",142
3,"LINESTRING (652818.194 9656290.952, 652800.275...",142
4,"LINESTRING (652798.904 9656319.503, 652778.911...",142
...,...,...
40656,"LINESTRING (655997.023 9707964.032, 656017.326...",155
40657,"LINESTRING (655995.058 9707989.473, 656017.001...",155
40658,"LINESTRING (656007.769 9708024.684, 656023.336...",155
40659,"LINESTRING (656375.925 9707430.346, 656375.925...",202


In [17]:
# Calcula o comprimento das linhas em metros
resultado_df['comp_metro'] = resultado_df['linha'].length
#resultado_df.drop(columns=['tamanho_metros'], inplace=True)
print(resultado_df)

                                                   linha  id_poligono  \
0      LINESTRING (652854.686 9656206.853, 652854.686...          142   
1      LINESTRING (652852.045 9656259.164, 652830.221...          142   
2      LINESTRING (652843.613 9656271.315, 652820.691...          142   
3      LINESTRING (652818.194 9656290.952, 652800.275...          142   
4      LINESTRING (652798.904 9656319.503, 652778.911...          142   
...                                                  ...          ...   
40656  LINESTRING (655997.023 9707964.032, 656017.326...          155   
40657  LINESTRING (655995.058 9707989.473, 656017.001...          155   
40658  LINESTRING (656007.769 9708024.684, 656023.336...          155   
40659  LINESTRING (656375.925 9707430.346, 656375.925...          202   
40660  LINESTRING (656375.763 9707441.654, 656371.987...          202   

       comp_metro  
0        0.000001  
1       28.636685  
2       30.078293  
3       23.188452  
4       25.002382  
...

In [19]:
# TIRANDO TODAS AS DISTANCIAS MAIORES QUE 300 METROS E MENORES QUE 10 METROS
resultado_df = resultado_df[(resultado_df['comp_metro'] >= 10) & (resultado_df['comp_metro'] <= 300)]

In [21]:
# Agrupa por 'id_poligono' e calcula média e mediana
estatisticas = resultado_df.groupby('id_poligono')['comp_metro'].agg(['mean', 'median']).reset_index()

In [23]:
# Renomeando as colunas
estatisticas.columns = ['ID', 'nav_media', 'nav_mediana']

In [25]:
# Realizando o merge com as Estatisticas
poligonos = poligonos.merge(estatisticas, on='ID', how='left')

In [27]:
# Calcular a área e adicionar como nova coluna
poligonos['area_km2'] = poligonos.geometry.area / 1e6  # Converte de m² para km²

# Exibindo o resultado
print(poligonos)

#poligonos.to_file('C:/Users/gus_h/Downloads/poligonos_fase1.shp')

      CD_MUN NM_MUN SIGLA_UF   ID  \
0    1501204  Baião       PA    0   
1    1501204  Baião       PA    1   
2    1501204  Baião       PA    2   
3    1501204  Baião       PA    3   
4    1501204  Baião       PA    4   
..       ...    ...      ...  ...   
615  1501204  Baião       PA  615   
616  1501204  Baião       PA  616   
617  1501204  Baião       PA  617   
618  1501204  Baião       PA  618   
619  1501204  Baião       PA  619   

                                              geometry  nav_media  \
0    POLYGON ((654069.692 9611078.066, 654068.268 9...  70.814157   
1    POLYGON ((640683.564 9664624.821, 640728.876 9...        NaN   
2    POLYGON ((640767.034 9664727.257, 640762.498 9...        NaN   
3    POLYGON ((640411.055 9665112.566, 640413.404 9...        NaN   
4    POLYGON ((640152.799 9665481.197, 640113.646 9...        NaN   
..                                                 ...        ...   
615  POLYGON ((713623.633 9799976.032, 713587.844 9...        NaN   
616

In [29]:
# Criar uma cópia do DataFrame para evitar modificações no original
poligonos_copy = poligonos.copy()

In [31]:
# Iterar sobre as linhas
for index, row in poligonos_copy.iterrows():
    # Verifica se a linha atual possui um valor válido em 'area_km2'
    if pd.notna(row['area_km2']):
        # Cria um DataFrame com linhas onde 'area_km2' não é nulo, excluindo a linha atual
        closest_row = poligonos_copy.loc[(poligonos_copy['area_km2'].notna())&(poligonos_copy.index != index)].copy()
    
        # Calcula a diferença absoluta com as áreas existentes
        closest_row['area_diff'] = (closest_row['area_km2'] - row['area_km2']).abs()
        closest_rows = closest_row.sort_values(by='area_diff')

        # Itera sobre as linhas mais próximas até encontrar valores válidos para 'nav_media' e 'nav_mediana'
        for _, closest_row in closest_rows.iterrows():        
            if pd.notna(closest_row['nav_media']) and pd.notna(closest_row['nav_mediana']):
                 # Atribui valores válidos encontrados
                poligonos_copy.at[index, 'nav_media'] = closest_row['nav_media']
                poligonos_copy.at[index, 'nav_mediana'] = closest_row['nav_mediana']
                break

In [33]:
# Exibindo o resultado atualizado
print(poligonos_copy)

      CD_MUN NM_MUN SIGLA_UF   ID  \
0    1501204  Baião       PA    0   
1    1501204  Baião       PA    1   
2    1501204  Baião       PA    2   
3    1501204  Baião       PA    3   
4    1501204  Baião       PA    4   
..       ...    ...      ...  ...   
615  1501204  Baião       PA  615   
616  1501204  Baião       PA  616   
617  1501204  Baião       PA  617   
618  1501204  Baião       PA  618   
619  1501204  Baião       PA  619   

                                              geometry   nav_media  \
0    POLYGON ((654069.692 9611078.066, 654068.268 9...   79.563334   
1    POLYGON ((640683.564 9664624.821, 640728.876 9...  119.048431   
2    POLYGON ((640767.034 9664727.257, 640762.498 9...   78.112054   
3    POLYGON ((640411.055 9665112.566, 640413.404 9...   36.008604   
4    POLYGON ((640152.799 9665481.197, 640113.646 9...  132.994920   
..                                                 ...         ...   
615  POLYGON ((713623.633 9799976.032, 713587.844 9...   32.95666

In [53]:
# Aplica o buffer de acordo com o valor em nav_median e armazena em uma nova coluna
poligonos_copy['buffered_geometry'] = poligonos_copy.geometry.buffer(poligonos_copy['nav_mediana'])

# Substituir a geometria original pelo buffer
poligonos_copy['geometry'] = poligonos_copy['buffered_geometry']

# Remover a coluna de geometria extra
poligonos_copy = poligonos_copy.drop(columns=['buffered_geometry'])

In [55]:
print(poligonos_copy)

      CD_MUN NM_MUN SIGLA_UF   ID  \
0    1501204  Baião       PA    0   
1    1501204  Baião       PA    1   
2    1501204  Baião       PA    2   
3    1501204  Baião       PA    3   
4    1501204  Baião       PA    4   
..       ...    ...      ...  ...   
615  1501204  Baião       PA  615   
616  1501204  Baião       PA  616   
617  1501204  Baião       PA  617   
618  1501204  Baião       PA  618   
619  1501204  Baião       PA  619   

                                              geometry   nav_media  \
0    POLYGON ((633957.913 9598945.849, 633814.563 9...   79.563334   
1    POLYGON ((640683.842 9664862.008, 640686.949 9...  119.048431   
2    POLYGON ((640755.472 9664543.224, 640741.577 9...   78.112054   
3    POLYGON ((640332.626 9664955.484, 640330.534 9...   36.008604   
4    POLYGON ((640334.914 9665275.754, 640330.209 9...  132.994920   
..                                                 ...         ...   
615  POLYGON ((713563.273 9799924.796, 713547.348 9...   32.95666

In [57]:
poligonos_copy.to_file('C:/Users/gus_h/Downloads/poligonos_copy.shp')

C:\Users\gus_h\AppData\Local\Temp\ipykernel_27188\1614982787.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  poligonos_copy.to_file('C:/Users/gus_h/Downloads/poligonos_copy.shp')
C:\Users\gus_h\anaconda3\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'nav_mediana' to 'nav_median'
  ogr_write(
